### Installing Dependencies

In [2]:
!powershell -Command "pip install -r requirements.txt"

In [3]:
from gensim.models import Word2Vec
import json
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import openai
import pandas as pd
import openai
from transformers import BertModel, BertForMaskedLM, BertTokenizer
import torch
import json

c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution\env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
%cd Exploiting-Food-Embeddings-for-Ingredient-Substitution/

[WinError 2] The system cannot find the file specified: 'Exploiting-Food-Embeddings-for-Ingredient-Substitution/'
c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution


### Evaluation of Food2Vec Model

In [5]:
## Downloading the Food2vec model

## For Linux
# !wget https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/food2vec_models.zip

# For Windows
!powershell -Command "Invoke-WebRequest -Uri 'https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/food2vec_models.zip' -OutFile 'food2vec_models.zip'"

In [6]:
## For Linux
# !unzip -qq food2vec_models.zip
# !mv food2vec_models/* food2vec/models/
# !rm -r food2vec_models

# For Windows
!powershell -Command "Expand-Archive -Path 'food2vec_models.zip' -DestinationPath 'food2vec_models'"
!powershell -Command "Move-Item 'food2vec_models\*' 'food2vec\models\'"
!powershell -Command "Remove-Item 'food2vec_models' -Recurse -Force"

Move-Item : Cannot create a file when that file already exists.
At line:1 char:1
+ Move-Item 'food2vec_models\*' 'food2vec\models\'
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: (C:\Users\hp\Emb...food2vec_models:DirectoryInfo) [Move-Item], IOException
    + FullyQualifiedErrorId : MoveDirectoryItemIOError,Microsoft.PowerShell.Commands.MoveItemCommand
 


In [7]:
with open('foodbert/data/used_ingredients.json', 'r') as f:
    used_ingredients = json.load(f)

In [8]:
data = pd.read_csv("C:/Users/hp\Downloads/Ingredients Dinnerfy Airtable.csv")

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2038 entries, 0 to 2037
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Name     2037 non-null   object
 1   Recipes  1122 non-null   object
 2   look     2038 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 47.9+ KB


In [10]:
my_ingredients = list(data["Name"].astype("str"))
# print(my_ingredients)
len(my_ingredients)

2038

In [11]:
map_obj = map(lambda x: x.lower(), my_ingredients)

In [12]:
my_ingredients = list(map_obj)
# print(my_ingredients)
len(my_ingredients)

2038

In [13]:
filter_obj = filter(lambda x: x in used_ingredients, my_ingredients)

In [14]:
# Number of Dinnerfy ingredients in the ingredients vocabulary of Food2Vec Model

len(list(filter_obj))

664

In [15]:
# Loading the food2vec model 
model = Word2Vec.load('food2vec/models/food2vec_models/model.bin')

In [16]:
# Length of Food2Vec Embeddings is 100
len(model.wv["salt"])

100

In [17]:
w1 = ["kosher"]
model.wv.most_similar (positive=w1,topn=6)

[('Kosher', 0.6294300556182861),
 ('cracked_black_pepper', 0.6071157455444336),
 ('Fresh-', 0.5951094031333923),
 ('dried_parsley', 0.5932490229606628),
 ('maldon', 0.5932028293609619),
 ('morton', 0.5621879696846008)]

In [18]:
print(f"Cosine Similarity between salt and kosher_salt - {model.wv.similarity(w1='salt', w2 ='kosher_salt')}")
print(f"Cosine Similarity between salt and pepper - {model.wv.similarity(w1='salt', w2 ='pepper')}")

Cosine Similarity between salt and kosher_salt - 0.8143872618675232
Cosine Similarity between salt and pepper - 0.18186870217323303


In [19]:
print(f"Cosine Similarity between parmesan_cheese and grated_parmesan_cheese - {model.wv.similarity(w1='parmesan_cheese', w2 ='grated_parmesan_cheese')}")
print(f"Cosine Similarity between parmesan_cheese and mozzarella_cheese - {model.wv.similarity(w1='parmesan_cheese', w2 ='mozzarella_cheese')}")

Cosine Similarity between parmesan_cheese and grated_parmesan_cheese - 0.7766836285591125
Cosine Similarity between parmesan_cheese and mozzarella_cheese - 0.8087217211723328


In [20]:
model.wv.most_similar("parmesan_cheese", topn = 5)

[('parmesan', 0.9504023790359497),
 ('romano_cheese', 0.8272979259490967),
 ('mozzarella_cheese', 0.8087217211723328),
 ('cheddar_cheese', 0.7970229983329773),
 ('parmigiano', 0.780430257320404)]

In [21]:
print(f"Cosine Similarity between garlic and garlic_clove - {model.wv.similarity(w1='garlic', w2 ='garlic_clove')}")
print(f"Cosine Similarity between garlic and ginger - {model.wv.similarity(w1='garlic', w2 ='ginger')}")

Cosine Similarity between garlic and garlic_clove - 0.6446242332458496
Cosine Similarity between garlic and ginger - 0.5138317346572876


In [22]:
model.wv.most_similar("garlic", topn = 5)

[('minced_garlic', 0.8553013205528259),
 ('crushed_garlic', 0.8153051137924194),
 ('chopped_garlic', 0.7908270955085754),
 ('garlic_paste', 0.7144184708595276),
 ('garlic;', 0.697868287563324)]

### Food2Vec Model

#### Advantages:
- Generates highly representative embeddings for ingredients in its vocab

#### Limitations:
- Biggest Limitation - cannot find embeddings for any other ingredient other than the ingredients in the vocab

### Evaluation OpenAI's Embeddings of Ingredients

In [23]:
# Function for getting the embeddings of ingredients from OpenAI Embeddings Model
def get_embeddings(input):
    response = openai.Embedding.create(
        input= input,
        model="text-embedding-ada-002"
    )
    embeddings = response['data'][0]['embedding']

    return np.array(embeddings)

In [24]:
# Function to evaluate the similarity of two embeddings

def similarity(w1, w2):
    similarity = cosine_similarity(w1.reshape(1, -1), w2.reshape(1, -1))
    return f"Cosine Similarity: {similarity[0][0]}"

In [25]:
salt = get_embeddings("salt")
kosher_salt = get_embeddings("kosher_salt")
pepper= get_embeddings("pepper")

In [26]:
# OpenAI makes embeddings of dimension 1536
len(salt)

1536

In [27]:
print(f"Salt and Pepper - {similarity(salt, pepper)}")
print(f"Salt and Kosher Salt - {similarity(salt, kosher_salt)}")

Salt and Pepper - Cosine Similarity: 0.8295305364447769
Salt and Kosher Salt - Cosine Similarity: 0.8551105468777112


Salt and Kosher Salt have a high cosine similarity but salt and pepper also have a reasonably high cosine similarity and their is only a marginal difference between the two cosine similarities.

In [29]:
parmesan  = get_embeddings("parmesan_cheese")
grated_parmesan  = get_embeddings("grated_parmesan_cheese")
mozzarella = get_embeddings("mozzarella_cheese")

In [30]:
print(f"Parmesan Cheese and Grated Parmesan Cheese - {similarity(parmesan, grated_parmesan)}")
print(f"Parmesan Cheese and Mozzarella Cheese - {similarity(parmesan, mozzarella)}")

Parmesan Cheese and Grated Parmesan Cheese - Cosine Similarity: 0.9385202933225403
Parmesan Cheese and Mozzarella Cheese - Cosine Similarity: 0.9329625437248483


Parmesan Cheese's cosine similarity with grated parmesan cheese is almost equal to its cosine similarity with mozzarella cheese, however, the cosine similarity with grated parmesan cheese should have been more since its just a different form of the same ingredient.

In [ ]:
garlic  = get_embeddings("garlic")
garlic_clove  = get_embeddings("garlic_clove")
ginger = get_embeddings("ginger")

In [ ]:
print(f"Garlic and Garlic Cloves - {similarity(garlic, garlic_clove)}")
print(f"Garlic and Ginger - {similarity(garlic, ginger)}")

Garlic and Garlic Cloves - Cosine Similarity: 0.9181282821028365
Garlic and Ginger - Cosine Similarity: 0.8690620962452782


### Observation on OpenAI Embeddings

The embeddings are representative of the strong similarity between any two similar ingredients. However, they tend to have a high cosine similarity even between ingredients which are not similar, which makes it difficult to identify the most similar ingredients and different names of the same ingredient.

### Evaluation of FoodBERT 

In [31]:
## Downloading the FoodBERT model (Can take approx. 30 mins to run)

## For Linux
# !wget https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/foodbert_data.zip

## For Windows
!powershell -Command "Invoke-WebRequest -Uri 'https://github.com/ChantalMP/Exploiting-Food-Embeddings-for-Ingredient-Substitution/releases/download/0.1/foodbert_data.zip' -OutFile 'foodbert_data.zip'"

In [32]:
## For Linux
# !unzip -qq foodbert_data.zip
# !mv foodbert_data/* foodbert/data/
# !rm -r foodbert_data

# For Windows
!powershell -Command "Expand-Archive -Path 'foodbert_data.zip' -DestinationPath 'foodbert_data'"
!powershell -Command "Move-Item 'foodbert_data\*' 'foodbert\data\'"
!powershell -Command "Remove-Item 'foodbert_data' -Recurse -Force"


Move-Item : Cannot create a file when that file already exists.
At line:1 char:1
+ Move-Item 'foodbert_data\*' 'foodbert\data\'
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: (C:\Users\hp\Emb...a\foodbert_data:DirectoryInfo) [Move-Item], IOException
    + FullyQualifiedErrorId : MoveDirectoryItemIOError,Microsoft.PowerShell.Commands.MoveItemCommand
 
Move-Item : Cannot create a file when that file already exists.
At line:1 char:1
+ Move-Item 'foodbert_data\*' 'foodbert\data\'
+ ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~
    + CategoryInfo          : WriteError: (C:\Users\hp\Emb...t_data\__MACOSX:DirectoryInfo) [Move-Item], IOException
    + FullyQualifiedErrorId : MoveDirectoryItemIOError,Microsoft.PowerShell.Commands.MoveItemCommand
 


In [33]:
!pip3 install torch torchvision torchaudio

In [34]:
with open('foodbert/data/used_ingredients.json', 'r') as f:
    used_ingredients = json.load(f)
tokenizer = BertTokenizer(vocab_file='foodbert/data/bert-base-cased-vocab.txt', do_lower_case=False, max_len=128, never_split=used_ingredients)

In [35]:
model = BertModel.from_pretrained(pretrained_model_name_or_path='foodbert/data/foodbert_data/mlm_output/checkpoint-final')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(33247, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [106]:
def get_bert_embedding(word):
    """
    Returns the average of the token indices except the last (EOS) and the first token (BOS)
    """
    if word:
        input_ids = torch.tensor(tokenizer.encode(word, add_special_tokens=True)).unsqueeze(0)
        input_ids = input_ids.to(device)
        outputs = model(input_ids)
        embedding = np.squeeze(np.sum(outputs[0].detach().numpy()[:,0:-1,:],axis = 1)/(outputs[0].detach().numpy().shape[1] - 2))

        return embedding


In [109]:
# FoodBERT generates 768 dimensional embeddings for ingredients

get_bert_embedding("salt").shape


(768,)

In [110]:
salt = get_bert_embedding("salt")
kosher_salt = get_bert_embedding("kosher_salt")
pepper = get_bert_embedding("pepper")

print(f"Salt and Kosher Salt - {similarity(salt, kosher_salt)}")
print(f"Salt and Pepper - {similarity(salt, pepper)}")

Salt and Kosher Salt - Cosine Similarity: 0.8881312608718872
Salt and Pepper - Cosine Similarity: 0.6598259806632996


FoodBERT is clearly able to highlight the strong similarity between salt and kosher salt and also the low similarity between salt and pepper.

In [111]:
parmesan = get_bert_embedding("parmesan_cheese")
grated_parmesan = get_bert_embedding("grated_parmesan_cheese")
mozzarella = get_bert_embedding("mozzarella_cheese")

print(f"Parmesan Cheese and Grated Parmesan Cheese - {similarity(parmesan, grated_parmesan)}")
print(f"Parmesan Cheese and Mozzarella Cheese - {similarity(parmesan, mozzarella)}")

Parmesan Cheese and Grated Parmesan Cheese - Cosine Similarity: 0.8406903743743896
Parmesan Cheese and Mozzarella Cheese - Cosine Similarity: 0.8513528108596802


Embeddings for Parmesan Cheese and Grated Parmesan Cheese and parmesan cheese and mozzarella cheese have a high cosine similarity.

In [112]:
garlic = get_bert_embedding("garlic")
garlic_clove = get_bert_embedding("garlic_clove")
ginger = get_bert_embedding("ginger")

print(f"Garlic and Garlic Cloves - {similarity(garlic, garlic_clove)}")
print(f"Garlic and Ginger - {similarity(garlic, ginger)}")

Garlic and Garlic Cloves - Cosine Similarity: 0.8596292734146118
Garlic and Ginger - Cosine Similarity: 0.5114772915840149


Garlic and Garlic Cloves have a high cosine similarity which is valid, since they are synonymous, while the cosine similarity for Garlic and Ginger is low which is also valid.

In [113]:
potato = get_bert_embedding("potato")
sweet_potato = get_bert_embedding("sweet_potato")
sliced_potato = get_bert_embedding("sliced_potato")

print(f"Potato and Sliced Potato - {similarity(potato, sliced_potato)}")
print(f"Potato and Sweet Potato - {similarity(potato, sweet_potato)}")

Potato and Sliced Potato - Cosine Similarity: 0.8131145238876343
Potato and Sweet Potato - Cosine Similarity: 0.7819738984107971


Cosine Similarity between Potato and Sliced Potato is higher than the cosine similarity between Potato and Sweet Potato which is correct, since sliced potato is processed form of potatoes while sweet potato is a different ingredient.

In [114]:
Crushed_Red_Pepper_Flakes = get_bert_embedding("crushed_red_pepper_flakes")
Chilli_Flakes = get_bert_embedding("chilli_flakes")
Oregano_Seasoning = get_bert_embedding("oregano_seasoning")

print(f"Crushed_Red_Pepper_Flakes and Chilli Flakes - {similarity(Crushed_Red_Pepper_Flakes, Chilli_Flakes)}")
print(f"Crushed_Red_Pepper_Flakes and Oregano Seasoning - {similarity(Crushed_Red_Pepper_Flakes, Oregano_Seasoning)}")

Crushed_Red_Pepper_Flakes and Chilli Flakes - Cosine Similarity: 0.8620383739471436
Crushed_Red_Pepper_Flakes and Oregano Seasoning - Cosine Similarity: 0.7542827129364014


FoodBERT is able to generate apt embeddings for new ingredients like crushed_red_pepper flakes and also obtain the right relationship between new ingredients.

In [115]:
onion = get_bert_embedding("onion")
large_onion = get_bert_embedding("large_onion")

print(f"Onion and Large Onion - {similarity(onion, large_onion)}")

Onion and Large Onion - Cosine Similarity: 0.7472556233406067


Onion and Large Onion are the same ingredients, however, they have a relatively low degree of similarity

In [116]:
egg = get_bert_embedding("egg")
boiled_egg = get_bert_embedding("boiled_egg")
eggplant = get_bert_embedding("eggpant")

print(f"Egg and Boiled Egg - {similarity(egg, boiled_egg)}")
print(f"Egg and Eggplant - {similarity(egg, eggplant)}")

Egg and Boiled Egg - Cosine Similarity: 0.6770237684249878
Egg and Eggplant - Cosine Similarity: 0.7074657678604126


Egg and boiled egg are the same ingredients, however, their cosine similarity is low. On the other hand, egg and eggplant have a higher cosine similarity which are completely different food items.

In [117]:
potato = get_bert_embedding("potato")
potatoes = get_bert_embedding("potatoes")


print(f"Potato and Potatoes - {similarity(potato, potatoes)}")

Potato and Potatoes - Cosine Similarity: 0.6225559115409851


Potato and potatoes have a very low cosine similarity even though they are the same ingredients

In [118]:
tomato = get_bert_embedding("tomato")
cherry_tomato = get_bert_embedding("cherry_tomato")

print(f"Tomato and Cherry Tomato - {similarity(tomato, cherry_tomato)}")

Tomato and Cherry Tomato - Cosine Similarity: 0.7752974629402161


### Observations on FoodBERT Embeddings

#### Advantages:
- Able to generate reasonable embeddings for previously unseen ingredients 
- Performs adequately in differentiating between closely used ingredients (like salt and pepper)

#### Limitations:
- Performs poorly on plural form of the same ingredient
- All ingredients should be in the same form (like "FirstName_LastName") otherwise can get unexpected results
- Poor results with prefixes like "large" before the ingredient


#### Ways to Use
- Embeddings for a good database of ingredients can be generated and stored in a database and the unknown ingredients can be matched with the already existing database by calculating cosine similarity
- Prefixes like "large" can be removed from the ingredients can be removed 

### Creating a Vector Database using Pinecone

In [119]:
ingredients_df = pd.read_csv("C:/Users/hp/Downloads/Ingredients Dinnerfy Airtable.csv")

In [120]:
ingredients_df = ingredients_df.fillna("")
np.sum(ingredients_df.isna())

c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution\env\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Name       0
Recipes    0
look       0
dtype: int64

In [121]:
# Adding Embeddings Column to Dataframe

ingredients_df["embeddings"] = ingredients_df["Name"].apply(get_bert_embedding)
print(ingredients_df)

                     Name                                            Recipes  \
0                    Salt  Slow Cooker Whole Chicken,Mediterranean Pasta ...   
1            Black_Pepper  Lentil Soup,Salmon with Fennel,Greek Chicken W...   
2               Olive_Oil  Pan-Seared Salmon,Salmon with Citrus Sauce,Lem...   
3                  Garlic  Almond Dijon Chicken,Quinoa and Sausage-Stuffe...   
4                   Onion  Slow Cooker Whole Chicken,Quinoa and Sausage-S...   
...                   ...                                                ...   
2033     Red_chili_powder                                                      
2034  Ginger_garlic_paste                                                      
2035         Tomato_puree                                                      
2036        Doritos_Chips                                                      
2037          Egg_noodles                                                      

      look                             

In [250]:
ingredients_df = ingredients_df.dropna()
np.sum(ingredients_df.isna())

c:\Users\hp\Embeddings Project\Exploiting-Food-Embeddings-for-Ingredient-Substitution\env\Lib\site-packages\numpy\core\fromnumeric.py:86: FutureWarning: The behavior of DataFrame.sum with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return reduction(axis=axis, out=out, **passkwargs)


Name          0
embeddings    0
id            0
dtype: int64

In [175]:
ingredients_df["embeddings"] = ingredients_df["embeddings"].apply(lambda x: x.tolist())

C:\Users\hp\AppData\Local\Temp\ipykernel_15820\3857504543.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ingredients_df["embeddings"] = ingredients_df["embeddings"].apply(lambda x: x.tolist())


In [144]:
# ingredients_df = ingredients_df.drop("Recipes", axis = 1)

In [145]:
ingredients_df["id"] = ingredients_df.index

In [160]:
import pinecone

In [179]:
# Helper functions for upserting data to vector db

def chunker(seq, size):
 'Yields a series of slices of the original iterable, up to the limit of what size is.'
 for pos in range(0, len(seq), size):
   yield seq.iloc[pos:pos + size]
   

def convert_data(chunk):
 'Converts a pandas dataframe to be a simple list of tuples, formatted how the `upsert()` method in the Pinecone Python client expects.'
 data = []
 for i in chunk.to_dict('records'):
  if 'metadata' in i:
   data.append((str(i['id']), i['embeddings'], i['metadata']))
  else:
   data.append((str(i['id']), i['embeddings']))
 return data

In [189]:
pinecone.init(
    api_key="84d086bd-a563-4439-8d86-7218de523a49",
    environment="gcp-starter"
)
# check if 'ingredient-search' index already exists (only create index if not)
# if 'bgb' not in pinecone.list_indexes():
#     pinecone.create_index('ingredient-search', dimension = 768, metric = "cosine")
# # connect to index
index = pinecone.Index('ingredient-search')


In [ ]:
# Adding data to pinecone vector db
for chunk in chunker(ingredients_df,100):
  index.upsert(vectors=convert_data(chunk))

In [276]:
# Funtion to find the most similar ingredients to an ingredient
def similar_ingredients(query, top_k = 5):
    xq = get_bert_embedding(query).tolist()

    xc = index.query(xq, top_k = top_k, include_metadata = False, metric = "cosine")
    match_indices= []
    match_scores = []
    for match in xc["matches"]:
        match_indices.append(match["id"])
        match_scores.append(match["score"])

    return list(zip([ingredients_df.loc[int(id)]["Name"] for id in match_indices], match_scores))

In [277]:
similar_ingredients("salt", 5)

[('Salt', 0.923648894),
 ('taco_seasoning', 0.685300231),
 ('Kosher_salt', 0.6826213),
 ('fresh_lemon_juice', 0.682364643),
 ('Salt_2', 0.670577645)]

In [280]:
similar_ingredients("flour")

[('Flour', 0.652291477),
 ('All-purpose_flour', 0.605266),
 ('Whole_wheat_flour', 0.584830761),
 ('flour_tortillas', 0.584145427),
 ('Whole-wheat_flour', 0.582782388)]

In [282]:
similar_ingredients("garlic_cloves")

[('garlic_cloves', 0.999789238),
 ('Peeled_garlic_cloves', 0.914329469),
 ('Minced_garlic_cloves', 0.912463486),
 ('Minced_garlic_(3_cloves)', 0.876065075),
 ('Minced_garlic', 0.800927937)]

In [283]:
similar_ingredients("shredded_onion")

[('Shredded_Lettuce', 0.920199454),
 ('Shredded_cheese', 0.920153856),
 ('Shredded_Chicken', 0.917511582),
 ('Shredded_cheddar_cheese', 0.908815682),
 ('Shredded_Cheese', 0.906285524)]

Observation: Gives more weightage to the processing rather than the ingredient in this case

In [290]:
similar_ingredients("Chilli Flakes")

[('Chili_Flakes', 0.910840809),
 ('Red_Chili_Flakes', 0.816604435),
 ('Celery_Stalks', 0.785562336),
 ('Red Pepper Flakes', 0.783036947),
 ('Saffron_Threads', 0.782991171)]

In [294]:
# Almost all wrong top 5
similar_ingredients("Pepper")

[('Sage', 0.999222398),
 ('Pepper', 0.999049425),
 ('Bacon', 0.998535097),
 ('Honey', 0.998523772),
 ('Water', 0.998472)]

In [296]:
similar_ingredients("Extremely thinly sliced green onion")

[('Thinly_sliced_green_onion', 0.89614141),
 ('Thinly_sliced_red_onion', 0.878154695),
 ('Thinly_sliced_Italian_prosciutto', 0.841239154),
 ('Sliced_green_onions', 0.817920446),
 ('Medium-diced_fennel', 0.811104476)]

In [297]:
similar_ingredients("kosher salt")

[('Kosher_salt', 0.80768311),
 ('Seasoned_salt', 0.745249152),
 ('Kosher_Salt_2', 0.718404293),
 ('Kosher_Salt', 0.714230478),
 ('Garlic_salt', 0.707502604)]